Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menu bar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menu bar, select Cell$\rightarrow$Run All).

Make sure that in addition to the code, you provide written answers for all questions of the assignment. 

Below, please fill in your name and collaborators:

In [1]:
NAME = "Anurag Singh Negi"
COLLABORATORS = ""

## Assignment 2 - Data Analysis using Pandas
**(15 points total)**

For this assignment, we will analyze the open dataset with data on the passengers aboard the Titanic.

The data file for this assignment can be downloaded from Kaggle website: https://www.kaggle.com/c/titanic/data, file `train.csv`. It is also attached to the assignment page. The definition of all variables can be found on the same Kaggle page, in the Data Dictionary section.

Read the data from the file into pandas DataFrame. Analyze, clean and transform the data to answer the following question: 

**What categories of passengers were most likely to survive the Titanic disaster?**

**Question 1.**  _(4 points)_
* The answer to the main question - What categories of passengers were most likely to survive the Titanic disaster? _(2 points)_
* The detailed explanation of the logic of the analysis _(2 points)_

**Question 2.**  _(3 points)_
* What other attributes did you use for the analysis? Explain how you used them and why you decided to use them. 
* Provide a complete list of all attributes used.

**Question 3.**  _(3 points)_
* Did you engineer any attributes (created new attributes)? If yes, explain the rationale and how the new attributes were used in the analysis?
* If you have excluded any attributes from the analysis, provide an explanation why you believe they can be excluded.

**Question 4.**  _(5 points)_
* How did you treat missing values for those attributes that you included in the analysis (for example, `age` attribute)? Provide a detailed explanation in the comments.


In [17]:
import pandas as pd
import numpy as np
trains = pd.read_csv("train.csv")
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Creating a new column Salutation by extracting the Salutation from the Name column for each of the Passenger

trains['Salutation'] = trains['Name'].replace(regex=r'^.*,', value='').replace(regex=r'\..*$', value='').str.strip()

# For the passengers with the missing age we are assigning the median age of the Salutation of the group. For example
# if the person with Salutation Mr is missing the age column the median age for the Salutation of Mr will be assigned to it

trains['Age'] = round(trains.groupby('Salutation')['Age'].transform(lambda group: group.fillna(group.median())), 1)

# Creating a new group for the Fare group which has the groups created and assigned for 
# each 50 Dollars interval starting from 0 to 550

trains['Fare'] = trains['Fare'].astype(np.int64)
trains['Fare_group'] = pd.cut(trains['Fare'], bins=np.linspace(0, 550, 12))

# For the passengers missing the Cabin or the Embarked value we are assigning the values which have the maximum occurance

trains.fillna({'Cabin' : trains['Cabin'].value_counts().idxmax(), 'Embarked' :  trains['Embarked'].value_counts().idxmax()}, inplace=True)

# Creating a new Column for the Age group which has the groups created and assigned for 
# each 10 years interval starting from 0 to 80

trains['Age_group'] = pd.cut(trains['Age'], bins=np.linspace(0, 80, 9))

# Creating a new group for the Fare group which has the groups created and assigned for 
# each 50 Dollars interval starting from 0 to 550

trains['Fare'] = trains['Fare'].astype(np.int64)
trains['Fare_group'] = pd.cut(trains['Fare'], bins=np.linspace(0, 550, 12))
trains.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Salutation,Fare_group,Age_group
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7,C23 C25 C27,S,Mr,"(0.0, 50.0]","(20.0, 30.0]"
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71,C85,C,Mrs,"(50.0, 100.0]","(30.0, 40.0]"
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7,C23 C25 C27,S,Miss,"(0.0, 50.0]","(20.0, 30.0]"
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53,C123,S,Mrs,"(50.0, 100.0]","(30.0, 40.0]"
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8,C23 C25 C27,S,Mr,"(0.0, 50.0]","(30.0, 40.0]"


In [3]:
# ANSWER 1 Part 1: FEMALES IN 1st CLASS - From the analysis of the data it seems the survival of the passengers was mainly 
#                  based on their Sex, Passenger Class, Fare paid and Age. As the survival rate of the females passengers 
#                  and passengers who are in 1st class (also means paid higher fare) had the higher survival rate so 
#                  the category of passenger which are most likely to survive are FEMALES IN 1st CLASS with a survival 
#                  percentage of around 96% followed by women in second class with survival percentage of 92%.
#                  
#                  
# ANSWER 1 Part 2:Below are the details for stats and logic used to determine that FEMALES IN 1st CLASS have
#                  highest survival rate
#
#          Point 1) Pclass and Sex 
#
#          Point 2) Age and Sex 
# 
#          Point 3) Fare and Sex

In [4]:
# Point 1) Pclass and Sex : The survival rate of the passengers was impacted by the travel class of the passengers.
#                            Passengers travelling in the 1st class had higher chance of survival compared to passengers
#                            travelling in 2nd and 3rd class similarly passenegrs in 2nd Class had higher survival chances
#                            than the passenger in the 3rd class for both Male and Females. The Females in the 1st class 
#                            have the highest survival rate among all.


result_pclass_group = (pd.concat([ trains.groupby(['Pclass','Sex']).count().dropna()['PassengerId'], trains[trains['Survived'] == 1].groupby(['Pclass','Sex']).count().dropna()['PassengerId'], round(trains[trains['Survived'] == 1].groupby(['Pclass','Sex']).count().dropna()['PassengerId']* 100/trains.groupby(['Pclass','Sex']).count().dropna()['PassengerId'],2)], axis=1))
result_pclass_group.columns = [' Total_Passengrs' ,'Num_of_Survived_Passengers', "Percentage_of_Survived_Passengers"]
result_pclass_group

Total_Passengrs  Num_of_Survived_Passengers  Percentage_of_Survived_Passengers
Pclass Sex                                                                                    
1      female                94                          91                              96.81
       male                 122                          45                              36.89
2      female                76                          70                              92.11
       male                 108                          17                              15.74
3      female               144                          72                              50.00
       male                 347                          47                              13.54

In [5]:
# Point 2) Age and Sex :From the analysis of the data and grouping of the data based on the sex for
#                       each of the Age groups we can clearly see that the survival rate of the females is 
#                       much higher compared to the males. Females survival rate for all the age groups is higher
#                       than 60% while for males only kids(age 0-10) have a survival percentage of 56% 
#                       for rest of the groups it is in lower either in lower twenties or less than that.

result_age_group = (pd.concat([ trains.groupby(['Sex','Age_group']).count().dropna()['PassengerId'], trains[trains['Survived'] == 1].groupby(['Sex','Age_group']).count().dropna()['PassengerId'], round(trains[trains['Survived'] == 1].groupby(['Sex','Age_group']).count().dropna()['PassengerId']* 100/trains.groupby(['Sex','Age_group']).count().dropna()['PassengerId'],2)], axis=1))
result_age_group.columns = [' Total_Passengrs' ,'Num_of_Survived_Passengers', "Percentage_of_Survived_Passengers"]
result_age_group

Total_Passengrs  Num_of_Survived_Passengers  Percentage_of_Survived_Passengers
Sex    Age_group                                                                                    
female (0.0, 10.0]               31.0                        19.0                              61.29
       (10.0, 20.0]              46.0                        34.0                              73.91
       (20.0, 30.0]             117.0                        83.0                              70.94
       (30.0, 40.0]              72.0                        60.0                              83.33
       (40.0, 50.0]              31.0                        21.0                              67.74
       (50.0, 60.0]              14.0                        13.0                              92.86
       (60.0, 70.0]               3.0                         3.0                             100.00
male   (0.0, 10.0]               37.0                        21.0                              56.76
       (10.0, 20.0]              69.0                        10.0                              14.49
       (20.0, 30.0]             268.0                        37.0                              13.81
       (30.0, 40.0]             100.0                        23.0                              23.00
       (40.0, 50.0]              56.0                        12.0                              21.43
       (50.0, 60.0]              28.0                         4.0                              14.29
       (60.0, 70.0]              14.0                         1.0                               7.14
       (70.0, 80.0]               5.0                         1.0                              20.00

In [6]:
# Point 3) Fare and Sex:  It seems the people survival rate of the people who paid the high fare is much higher compared 
#                        to the people in the lower fare range (fare range 0-50 survival rate of around 32%) and we  
#                         also see that the passegnegrs who paid fare higher than 200 Dollars all survived but as their
#                          number is only 14 we prefer to use the group of Females in 1st Class as our choice of Category
#                          to most likely survive the Titanic disaster

result_pclass_group = (pd.concat([ trains.groupby(['Fare_group','Sex']).count().dropna()['PassengerId'], trains[trains['Survived'] == 1].groupby(['Fare_group','Sex']).count().dropna()['PassengerId'], round(trains[trains['Survived'] == 1].groupby(['Fare_group','Sex']).count().dropna()['PassengerId']* 100/trains.groupby(['Fare_group','Sex']).count().dropna()['PassengerId'],2)], axis=1))
result_pclass_group.columns = [' Total_Passengrs' ,'Num_of_Survived_Passengers', "Percentage_of_Survived_Passengers"]
result_pclass_group.dropna()

Total_Passengrs  Num_of_Survived_Passengers  Percentage_of_Survived_Passengers
Fare_group     Sex                                                                                    
(0.0, 50.0]    female             227.0                       151.0                              66.52
               male               490.0                        81.0                              16.53
(50.0, 100.0]  female              53.0                        50.0                              94.34
               male                53.0                        20.0                              37.74
(100.0, 150.0] female              15.0                        15.0                             100.00
               male                 9.0                         4.0                              44.44
(150.0, 200.0] female               7.0                         5.0                              71.43
               male                 2.0                         1.0                              50.00
(200.0, 250.0] female               7.0                         7.0                             100.00
(250.0, 300.0] female               4.0                         4.0                             100.00
(500.0, 550.0] female               1.0                         1.0                             100.00
               male                 2.0                         2.0                             100.00

In [7]:
#  ANSWER 2 Part 1 : I analysed all the attributed of the passengers but based on the analysis discarded some of 
#                    the attributes and used the attributes Survived, Sex, Fare, Name, Age and Pclass. 
#                    From the analysis of data we can see that the females have higher survival percentage 
#                    compared to males as a result of which I grouped the Fare, age and Pclass on Sex.
#                    As we have a wide raneg of Age and Fare I created additional columns for Fare_group 
#                    and Age_group to better understand their impact with the gender of the passenger.



In [8]:
#  ANSWER 2 Part 2 : Below is the list of all the attributes used
#                  1) Sex          : Gave us the detail on higher survival rate of females
#                  2) Pclass       : We see higher survival rate with highr passenger class
#                  3) Age          : used for Detail on the survival percentage with the Age 
#                  4) Name         : used to create a new field Salutation used for missing age values
#                  5) Fare         : Used to check on the impact of higher fares on the survival
#                  6) Fare_group   : Created to group different fare values for better analysis
#                  7) Age_group    : Created to group different age values for better analysis
#                  8) Salutation   : Created to get the missing ages for the passengers
#                  9) PassengerId  : Used as a count for passengers      

In [9]:
#  ANSWER 3 Part 1 : Below are the new attributes created and their details
#                  1) Salutation   : As I see a lot of passengers have missing age value I created a new field
#                                    by extracting the salutaion used in the name column for each of he passenger
#                                     which is then used to determine the missing age
#
#                  2) Fare_group   : As I see that there is a large variation in the fare of the passengers to better
#                                     understand the impact of the Fare on the Survival of the passenger I
#                                     decided to create additional column by breaking the fare in groups of 50 dollars apart.
#        
#                  3) Age_group    : As the age of the passengers showed a large number of different values I created 
#                                   the new column Age_group by breaking the age in groups of 10 years apart

trains[['Name','Salutation','Fare','Fare_group','Age', 'Age_group']]

,Name,Salutation,Fare,Fare_group,Age,Age_group
0,"Braund, Mr. Owen Harris",Mr,7,"(0.0, 50.0]",22.0,"(20.0, 30.0]"
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",Mrs,71,"(50.0, 100.0]",38.0,"(30.0, 40.0]"
2,"Heikkinen, Miss. Laina",Miss,7,"(0.0, 50.0]",26.0,"(20.0, 30.0]"
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",Mrs,53,"(50.0, 100.0]",35.0,"(30.0, 40.0]"
4,"Allen, Mr. William Henry",Mr,8,"(0.0, 50.0]",35.0,"(30.0, 40.0]"
5,"Moran, Mr. James",Mr,8,"(0.0, 50.0]",30.0,"(20.0, 30.0]"
6,"McCarthy, Mr. Timothy J",Mr,51,"(50.0, 100.0]",54.0,"(50.0, 60.0]"
7,"Palsson, Master. Gosta Leonard",Master,21,"(0.0, 50.0]",2.0,"(0.0, 10.0]"
8,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",Mrs,11,"(0.0, 50.0]",27.0,"(20.0, 30.0]"
9,"Nasser, Mrs. Nicholas (Adele Achem)",Mrs,30,"(0.0, 50.0]",14.0,"(10.0, 20.0]"


In [10]:
#  ANSWER 3 Part 2 : Below are the details of the discarded attributed

#Discarded attribute Cabin : The original data set only had the 204 passengers with the Cabin attribute
#                             which was a very small set to determine its impact on the survival of the 
#                             passengers
        
#Discarded attribute Embarked: The passenger point of embarkement didn't seem to have considerable impact on the
#                               survival of the passenger as we can see from the below data 
trains.groupby(['Survived','Embarked']).count()['PassengerId']          

Survived  Embarked
0         C            75
          Q            47
          S           427
1         C            93
          Q            30
          S           219
Name: PassengerId, dtype: int64

In [11]:
#Discarded attribute Sibsp: as I it was not cosiderably impacting the survival rate of the passenger as visible from the 
#                            below data

trains.groupby(['Survived','SibSp']).count()['PassengerId']          

Survived  SibSp
0         0        398
          1         97
          2         15
          3         12
          4         15
          5          5
          8          7
1         0        210
          1        112
          2         13
          3          4
          4          3
Name: PassengerId, dtype: int64

In [12]:
# I also checked if the married females had higher survival rate but as it was not considerably high it was 
#  not further explored
trains[trains['Salutation']=='Mrs'].groupby('Survived').count()['PassengerId']

Survived
0    26
1    99
Name: PassengerId, dtype: int64

In [13]:
# Discarded attribute Parch: No considerable impact on the attribute Parch on the survival of passenger
#                            Below is the data grouped by survived for Parch
trains.groupby(['Survived','Parch']).count()['PassengerId']          

Survived  Parch
0         0        445
          1         53
          2         40
          3          2
          4          4
          5          4
          6          1
1         0        233
          1         65
          2         40
          3          3
          5          1
Name: PassengerId, dtype: int64

In [14]:
# Discarded attribute Ticket: There are 681 unique values of the Ticket attribute so the impact of the tciket on
#                             the survival of the passenger will be random. 

trains['Ticket'].describe()

count        891
unique       681
top       347082
freq           7
Name: Ticket, dtype: object

In [15]:
#  ANSWER 4 : There are 3 attributes which are missing values Age, Cabin and Embarked

# Missing Age value : There are 177 passengers missing their Age in the data sheet.
#                      To fill the missing age value I decided to create a new column by extracting the Salutation of the
#                      Passengers from the Passenger name using regex. For the passengers missing the age the median age of the 
#                      Salutation of the passenger was assigned.                                  
#
#Below is the median of the various salutations for the passengers
trains.groupby('Salutation')['Age'].median()

Salutation
Capt            70.0
Col             58.0
Don             40.0
Dr              46.5
Jonkheer        38.0
Lady            48.0
Major           48.5
Master           3.5
Miss            21.0
Mlle            24.0
Mme             24.0
Mr              30.0
Mrs             35.0
Ms              28.0
Rev             46.5
Sir             49.0
the Countess    33.0
Name: Age, dtype: float64

In [16]:
# Missing Cabin and the Embarked : I assigned the most frequent value of the Cabin and the Embarked for the passengers 
#                                   Missing the Cabin and Embarked values. 

trains['Cabin'].value_counts().idxmax() , trains['Embarked'].value_counts().idxmax()

('C23 C25 C27', 'S')